## Library Import

In [1]:
import duckdb
import polars as pl
import dotenv
import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import load_setting
settings = load_setting()
import pandas as pd
from tools.redis_client import RedisTools

## Load Gold Data

In [ ]:
# Test connection to Gold Data parquet file
try:
    parquet_path = settings['process']['gold_path']
    print(f"Attempting to connect to: {parquet_path}/gold_picks.parquet")
    # Test Connection with a simple query
    pl_df = pl.read_parquet(f"../{parquet_path}/gold_picks.parquet")
    pl_df = pl_df.sort(["date"], descending=True)
    print("Connection Successful!")
    print(f"Test query result: {pl_df}")
    
except FileNotFoundError as e:
    print(f"File not found: {str(e)}")
except Exception as e:
    print(f"Connection Failed: {str(e)}")

Attempting to connect to: process/storage/gold/gold_picks.parquet
Connection Successful!
Test query result: shape: (1_256, 5)
┌────────────────────┬────────────────────┬────────────────────┬────────────────────┬──────────────┐
│ accelerating       ┆ long_accelerating  ┆ long_accumulating  ┆ velocity_maintaine ┆ date         │
│ ---                ┆ ---                ┆ ---                ┆ d                  ┆ ---          │
│ list[str]          ┆ list[str]          ┆ list[str]          ┆ ---                ┆ list[date]   │
│                    ┆                    ┆                    ┆ list[str]          ┆              │
╞════════════════════╪════════════════════╪════════════════════╪════════════════════╪══════════════╡
│ ["CBNK", "LZ", …   ┆ ["SEIC", "CPF", …  ┆ ["CPF", "LNKB", …  ┆ ["SO", "NC", …     ┆ [2025-06-27] │
│ "HSAI"]            ┆ "AZZ"]             ┆ "AZZ"]             ┆ "HWM"]             ┆              │
│ ["LRN", "GOOS", …  ┆ ["QRVO", "COMM", … ┆ ["QRVO", "KNSL", … ┆ [

In [ ]:
# Test connection to Gold Data parquet file
try:
    parquet_path = settings['process']['gold_path']
    print(f"Attempting to connect to: {parquet_path}/gold.parquet")
    # Test Connection with a simple query
    pl_df = pl.read_parquet(f"../{parquet_path}/gold.parquet")
    pl_df = pl_df.sort(["date"], descending=True)
    print("Connection Successful!")
    print(f"Test query result: {pl_df}")
    
except FileNotFoundError as e:
    print(f"File not found: {str(e)}")
except Exception as e:
    print(f"Connection Failed: {str(e)}")

Attempting to connect to: process/storage/gold/gold.parquet
Connection Successful!
Test query result: shape: (55_328_507, 5)
┌────────┬────────────┬──────────┬────────────────┬──────────────────────┐
│ symbol ┆ date       ┆ interval ┆ alert_type     ┆ signal               │
│ ---    ┆ ---        ┆ ---      ┆ ---            ┆ ---                  │
│ str    ┆ date       ┆ i32      ┆ str            ┆ str                  │
╞════════╪════════════╪══════════╪════════════════╪══════════════════════╡
│ A      ┆ 2025-06-27 ┆ 1        ┆ velocity_alert ┆ velocity_negotiating │
│ AA     ┆ 2025-06-27 ┆ 1        ┆ velocity_alert ┆ velocity_loss        │
│ AACB   ┆ 2025-06-27 ┆ 1        ┆ velocity_alert ┆ velocity_weak        │
│ AACG   ┆ 2025-06-27 ┆ 1        ┆ velocity_alert ┆ velocity_loss        │
│ AACT   ┆ 2025-06-27 ┆ 1        ┆ velocity_alert ┆ velocity_weak        │
│ …      ┆ …          ┆ …        ┆ …              ┆ …                    │
│ ZION   ┆ 2020-06-29 ┆ 13       ┆ ema_touch      

In [6]:
# Get the last row of the gold data
pl_df.head(1).select(['long_accelerating']).to_dicts()[0]['long_accelerating']

ColumnNotFoundError: long_accelerating

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF ["symbol", "date", "interval", "alert_type", ...]; PROJECT */5 COLUMNS

In [7]:
# Get all strategies from the gold data
pl_df.columns

['accelerating',
 'long_accelerating',
 'long_accumulating',
 'velocity_maintained',
 'date']

In [10]:
# Get the size of the gold data
# Get file size in bytes
file_size_bytes = os.path.getsize(f"../{settings['process']['gold_path']}/gold_picks.parquet")

# Convert to more readable format
def convert_size(size_bytes):
    """Convert bytes to human readable format"""
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024.0
    return f"{size_bytes:.2f} GB"

print(f"Gold data file size: {convert_size(file_size_bytes)}")
print(f"Number of rows: {len(pl_df)}")
print(f"Number of columns: {len(pl_df.columns)}")


Gold data file size: 2.92 MB
Number of rows: 1255
Number of columns: 5


## Cache to Redis

### Cache Stock of the Day

In [5]:
# Get the last row of the gold data
veggas_channel_pick  = pl_df.tail(1).select(["long_accelerating"]).to_dicts()[0]["long_accelerating"]

# Cache long-accelerating to Redis
redis_client = RedisTools()
redis_client.rpush("veggas_channel_pick", *veggas_channel_pick)

# Get long-accelerating from Redis
veggas_channel_pick = redis_client.lrange("veggas_channel_pick", 0, -1)
print(veggas_channel_pick)
print(len(veggas_channel_pick))

Successfully connected to Redis!
['BLK', 'CHT', 'VATE', 'VKTX', 'CHY', 'CI', 'CHWY', 'FMS', 'NVMI', 'ABT', 'DSGX', 'KEN', 'ZG', 'AMP', 'ALRM', 'AMD', 'META', 'VIRT', 'ZBRA', 'AAON', 'HASI', 'SCL', 'PRGO', 'DKL', 'TAL', 'SWTX', 'DQ', 'APDN', 'BILI', 'CLW', 'VEEV', 'DRS', 'MBOT', 'NVAX', 'JD', 'FITBI', 'SCPH', 'CHPT', 'ANGI', 'CLLS', 'AAPL', 'MHLD', 'HMC', 'FR', 'BL', 'VERI', 'SCNX', 'IZEA', 'JOUT', 'AMRC', 'SFIX', 'XYZ', 'POWI', 'AAMI', 'FI', 'YUMC', 'ALTS', 'APD', 'FFIV', 'DT', 'TAK', 'DGX', 'YEXT', 'SBAC', 'VHC', 'VIVK', 'BLD', 'FMC', 'AMAT', 'SCNI', 'FF', 'FOLD', 'VIR', 'BIPC', 'MDB', 'VIVS', 'MBUU', 'TEL', 'DOMO', 'AMPH', 'MGNX', 'MCRI', 'ABUS', 'PODD', 'SYRE', 'NTES', 'SCCO', 'DOCU', 'TER', 'TFX', 'FLWS', 'BMY', 'HLF', 'MCK', 'SGRY', 'BLFS', 'TEAM', 'ZWS', 'SDGR', 'PRAA', 'VRNS', 'POWW', 'AMT', 'DKNG', 'K', 'MEIP', 'AMCR', 'TBPH', 'DRD', 'AMH', 'BLDP', 'ZTO', 'PRI', 'BKYI', 'PLD', 'ZM', 'AAP', 'JNPR', 'ADBE', 'VC', 'XYLO', 'ACM', 'NTLA', 'CLWT', 'CLS', 'BMRN', 'ACN', 'JOYY', 'ANSS'